## Getting 2023 Weather Data from OpenWeatherMap API

### Import Necessary Libraries

In [ ]:
import pandas as pd
import requests
import pytz
from datetime import datetime

### Define the Constants

In [ ]:
PST_TIME_ZONE = pytz.timezone('America/Los_Angeles')            # Pacific Standard Time
API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"                    
LATITUDE, LONGITUDE = 37.585404, -122.407818                    # San Francisco latitude, longitude
START_TIMESTAMP, END_TIMESTAMP = 1674374741, 1704092400         # Start and end timestamps (for 2023)
REQUEST_INTERVAL = 7 * 24 * 60 * 60                             # OpenWeatherMap API allows 7 days per request
OUTPUT_CSV_PATH = '/path/to/your/output/folder/testing_one.csv'  

### Requesting from API (seven day limit per request)

In [ ]:
final_weather_data = pd.DataFrame()

# Loop through each time interval
current_start_timestamp = START_TIMESTAMP
while current_start_timestamp < END_TIMESTAMP:

    current_end_timestamp = min(current_start_timestamp + REQUEST_INTERVAL, END_TIMESTAMP)
    request_url = f"https://history.openweathermap.org/data/2.5/history/city?lat={LATITUDE}&lon={LONGITUDE}&type=hour&start={current_start_timestamp}&end={current_end_timestamp}&appid={API_KEY}"
    response = requests.get(request_url)


    if response.status_code == 200:
        # Parse the JSON response
        json_data = response.json()
        weather_data_list = json_data.get('list', [])

        # Convert the list of weather data into a Pandas DataFrame
        current_weather_data = pd.json_normalize(weather_data_list)
        current_weather_data['dt'] = pd.to_datetime(current_weather_data['dt'], unit='s', utc=True)
        current_weather_data['dt'] = current_weather_data['dt'].dt.tz_convert(PST_TIME_ZONE)  # Convert time to PST
        current_weather_data.set_index('dt', inplace=True)
        current_weather_data['weather_description'] = current_weather_data['weather'].apply(lambda x: x[0]['description'] if x else None)

        # Append the dataframe with each interval's data to the final dataframe
        final_weather_data = pd.concat([final_weather_data, current_weather_data])
        current_start_timestamp = current_end_timestamp
        
    else:
        error_message = f"Error fetching data: Status Code {response.status_code}, Response: {response.text}"      
        print(error_message)        
        break

# Save the final dataframe to a CSV file
final_weather_data.to_csv(OUTPUT_CSV_PATH)